In [1]:
# Initialization

import math
from datetime import datetime
import openpyxl as opxl
from openpyxl.utils.dataframe import dataframe_to_rows
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import model as ty
import proc as pr
import matplotlib.ticker as mtick
import sklearn.metrics

import scipy.constants as scc

import sys
sys.path.append('../RD-systems-and-test-benches')

import thermal.SK_standard as nm

import seaborn as sns

import utils.plot_functions as RD_pfun
import repo_config as rconfig

from IPython.core.display import HTML

import heat_transfer as bht

import fluids as fds
import ht 

import general as gen

import os

import scipy.integrate as integrate
import scipy.optimize as sco

import networkx as nx

import plotly.graph_objects as go

import plot_functions_here as pfun

import plotly.io as pio

from itertools import product
from itertools import chain

from tqdm.notebook import tqdm

import copy

from concurrent.futures import ProcessPoolExecutor

In [15]:
import repo_config as rconfig

import matplotlib
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick
from matplotlib.ticker import FuncFormatter
import matplotlib.gridspec as gridspec
import matplotlib_inline

import scienceplots

matplotlib_inline.backend_inline.set_matplotlib_formats('retina')
plt.style.use(['science'])

def to_percent(y, _):
    return f"{y * 100:.0f}\%"

# colors = ['olive', 'green', 'blue', 'orange', 'red', 'grey','yellow','bordeaux']
colors = ['olive','orange','blue','red','green','grey']
colors70 = [rconfig.get_hex_code(color, 70) for color in colors]
colors = [rconfig.get_hex_code(color, 100) for color in colors]
linestyles = ['-', '--',  (5, (10, 3)), '-.', (0, (1, 1)), (0, (3, 5, 1, 5)), (0, (3, 5, 1, 5)), (0, (3, 5, 1, 5))]
markers = ['o', 's', 'd', 'X', 'v', '^', 'P', 'H']

In [ ]:
folder_path = r'G:\Drive partagés\Cercle Hard\R&D\Modèles numériques PVT\PVT-perf-1Dmodel'
geometry_path = os.path.join(folder_path, 'Panels\\Panel_V4.1I.xlsx')
hypotheses_path = folder_path + '\\' + 'Hypotheses\\Model_hypotheses_calibrated_V4.411.xlsx'

# panelSpecs = pr.import_geometry(geometry_path)
# hyp = pr.create_dict_from_excel(hypotheses_path,'Main')

# steadyStateConditions_df = pd.read_excel(condi_path,header=2) ## Les deux première lignes ne sont pas à prendre en compte
# stepConditions = steadyStateConditions_df.to_dict(orient='records')

In [ ]:
recap = pd.read_excel(r'G:\Drive partagés\Cercle Hard\R&D\Thermal performance tests PVT\Recap_RD_TUV_SSI.xlsx')
recap = recap.drop(index=0)
recap.dropna(how='all', inplace=True)

groups = {key: group for key, group in recap.groupby('Design')}

In [ ]:
triplets = [
            {'design_name' : 'V4.411(NI)',
             'test_type' : 'SK'},
            {'design_name' : 'V4.411(NI)',
                'test_type' : 'Dark front'},
            {'design_name' : 'V4.412(F)',
                'test_type' : 'SK'},
            {'design_name' : 'V4.412(F)',
                'test_type' : 'Dark front'}
                ]

for triplet in triplets:
             
    triplet['panelSpecs'] = pr.import_geometry(os.path.join(f'{folder_path}\\Panels', f"Panel_{triplet['design_name']}.xlsx"))
        
    triplet['hyp'] = pr.create_dict_from_excel(hypotheses_path,'Main')
    triplet['steadyStateConditions_df'] = groups[triplet['design_name']].loc[groups[triplet['design_name']]['Type'] == triplet['test_type']].copy()
    pr.adjust_steadyStateConditions(triplet['steadyStateConditions_df'],triplet['hyp'],convert_celsius=True)

    triplet['steadyStateConditions_df'] = triplet['steadyStateConditions_df'].loc[(triplet['steadyStateConditions_df']['u'] != " ")]
    triplet['steadyStateConditions_df']['u'] = triplet['steadyStateConditions_df']['u'].astype(float)
    triplet['steadyStateConditions_df'] = triplet['steadyStateConditions_df'].loc[(triplet['steadyStateConditions_df']['u'] >= 0.1)]
    triplet['steadyStateConditions_df'] = triplet['steadyStateConditions_df'].loc[(triplet['steadyStateConditions_df']['Qdot'] > 25)]

    triplet['steadyStateConditions_df'].reset_index(drop=True,inplace=True)

### Implementation

In [ ]:
mapping_parameters_bounds = {

    'V4.411(NI)' : {
        'panelSpecs': {
            'flat' : {
                    'lambd_air' : (0.01e-3,2.e-3),
            }
        }
    },

    'V4.412(F)' : {
        'panelSpecs': {
            'flat' : {
                    'lambd_air' : (0.01e-3,2.e-3),
            }
        }
    },

    # 'V4.5F' : {
    #     'panelSpecs': {
    #         'flat' : {
    #                 'lambd_air' : (0.01e-3,2.e-3),
    #         }
    #     }
    # },

    'shared_designs' : {
        'panelSpecs': {
            'pv' : {
                    'tau_g' : (0.85, 0.95),
                    'eta_nom' : (0.21, 0.22),
                    # 'alpha_g' : (0.03,0.09),
                    # 'alpha_PV' : (0.87, 0.93),
                    # 'eps_PV' : (0.85, 0.95),
                    # 'eps_g' : (0.85, 0.95),
            },
            'flat' : {
                    # 'l_c' : (0.1e-3,0.6e-3),
                    'eps_tube' : (0.15,0.40),
                    # 'eps_ins' : (0.25,0.60),
            },
        }
    },

    'hyp': {
        # 'coeff_h_top_forced_range0' : (0.5, 3.),
        # 'coeff_h_top_forced_range1' : (0.5, 3.),
        # 'coeff_h_top_forced_range2' : (0.5, 3.),
    },
}

# Function to recursively flatten the parameter bounds
def flatten_parameters_bounds(d, parent_keys=[]):
    items = []
    for k, v in d.items():
        if isinstance(v, dict):
            items.extend(flatten_parameters_bounds(v, parent_keys + [k]))
        else:
            items.append((parent_keys + [k], v))
    return items

# Flatten the parameter bounds and extract parameter paths and bounds
flattened_parameters = flatten_parameters_bounds(mapping_parameters_bounds)
parameter_paths = [param[0] for param in flattened_parameters]
bounds = [param[1] for param in flattened_parameters]

mapping_parameters = {tuple(flattened_parameters[j][0]): j for j in range(len(flattened_parameters))}

In [ ]:
def flatten_dict(d, parent_key=()):
    """ Recursively flattens a nested dictionary. Keys become tuples of the path."""
    items = []
    for k, v in d.items():
        new_key = parent_key + (k,)
        if isinstance(v, dict):
            items.extend(flatten_dict(v, new_key).items())
        else:
            items.append((new_key, v))
    return dict(items)

In [ ]:
def update_specs(params, design, panelSpecs):
    for key in mapping_parameters_bounds[design]['panelSpecs']:
        if key in list(set(panelSpecs['decomp'].values())) + ['pv']:
            zones = list(panelSpecs['decomp'].keys()) + ['pv'] if key == 'pv' else [_ for _, value in panelSpecs['decomp'].items() if value == key]
            for z in zones:
                for subkey in mapping_parameters_bounds[design]['panelSpecs'][key]:
                    panelSpecs[z][subkey] = params[mapping_parameters[(design, 'panelSpecs', key, subkey)]]

                    if subkey == 'eps_tube':
                        for i in range(1,8):
                            panelSpecs[f'part{i}']['eps_tube'] = params[mapping_parameters[(design, 'panelSpecs', key, subkey)]]
                            panelSpecs[f'part{i}']['eps_abs'] = params[mapping_parameters[(design, 'panelSpecs', key, subkey)]]
        else:
            panelSpecs[key] = params[mapping_parameters[(design, 'panelSpecs', key)]]

def update_panelSpecs(params, design_name, panelSpecs):
    update_specs(params, design_name, panelSpecs)
    update_specs(params, 'shared_designs', panelSpecs)
    pr.update_panelSpecs_after_modif(panelSpecs)

def update_hyp(params, hyp):

    for key in mapping_parameters_bounds['hyp']:
        hyp[key] = params[mapping_parameters[('hyp', key)]]

In [ ]:
def simulation_function(params, triplet):

    panelSpecs = copy.deepcopy(triplet['panelSpecs'])
    hyp = copy.deepcopy(triplet['hyp'])
    steadyStateConditions_df = triplet['steadyStateConditions_df']

    update_panelSpecs(params, triplet['design_name'], panelSpecs)
    update_hyp(params, hyp)

    # Call the simulation function
    df_res, list_res = ty.simu_steadyStateConditions(panelSpecs, hyp, steadyStateConditions_df)
    return df_res


# Global counter for objective function calls
call_count = 0

# def callback_function(xk):
#     print(f"Iteration {len(iteration_history)}: params = {xk}")
#     iteration_history.append(xk.copy())

def objective_function(params, triplets):

    global call_count
    call_count += 1

    total_error = 0.0

    for triplet in triplets:

        df_res = simulation_function(params, triplet)

        # Compute the error
        error = np.sqrt(((df_res['Qdot_tube_fluid'] - triplet['steadyStateConditions_df']['Qdot']) ** 2).mean())
        total_error += error

    return total_error

def run_optimization(initial_guess, bounds, triplets):
    result = sco.minimize(
        objective_function,
        x0=initial_guess,
        args=(triplets,),  # Ensure you're passing the right triplet or list of triplets
        bounds=bounds,
        method='L-BFGS-B',
        options={'disp': True}
    )
    return result

### Differentiel evolution with intermediate solutions

In [ ]:
# Define a callback function to store intermediate results
intermediate_solutions = []

def callback(xk, convergence):
    intermediate_solutions.append(xk)

result = sco.differential_evolution(
    objective_function,
    bounds=bounds,
    args=(triplets,),
    strategy='best1bin',
    maxiter=1000,
    popsize=15,
    tol=0.01,
    callback=callback,
    polish=False  # Set to False to prevent local search polishing
)

In [ ]:
intermediate_solutions = [np.array([1.36401812e-04, 1.18639318e-04, 9.15453800e-01, 2.12979528e-01,
        1.34712524e-01]),
 np.array([1.81662646e-04, 9.00071190e-05, 9.21556484e-01, 2.12627471e-01,
        2.00967636e-01]),
 np.array([1.81662646e-04, 9.00071190e-05, 9.21556484e-01, 2.12627471e-01,
        2.00967636e-01]),
 np.array([1.81662646e-04, 9.00071190e-05, 9.21556484e-01, 2.12627471e-01,
        2.00967636e-01]),
 np.array([2.06112325e-04, 8.35401944e-05, 9.42724406e-01, 2.14523488e-01,
        1.39605081e-01]),
 np.array([2.21851741e-04, 8.01799426e-05, 9.48436095e-01, 2.12066913e-01,
        1.45063248e-01]),
 np.array([2.21851741e-04, 8.01799426e-05, 9.48436095e-01, 2.12066913e-01,
        1.45063248e-01]),
 np.array([2.21851741e-04, 8.01799426e-05, 9.48436095e-01, 2.12066913e-01,
        1.45063248e-01]),
 np.array([2.08331660e-04, 7.83145185e-05, 9.46826146e-01, 2.14456268e-01,
        2.74965993e-01]),
 np.array([1.97108210e-04, 6.67236976e-05, 9.49404370e-01, 2.15323647e-01,
        2.87280116e-01]),
 np.array([2.03859512e-04, 6.16802333e-05, 9.49072235e-01, 2.10828955e-01,
        3.14759949e-01]),
 np.array([2.03859512e-04, 6.16802333e-05, 9.49072235e-01, 2.10828955e-01,
        3.14759949e-01]),
 np.array([2.03859512e-04, 6.16802333e-05, 9.49072235e-01, 2.10828955e-01,
        3.14759949e-01]),
 np.array([2.03859512e-04, 6.16802333e-05, 9.49072235e-01, 2.10828955e-01,
        3.14759949e-01]),
 np.array([2.03859512e-04, 6.16802333e-05, 9.49072235e-01, 2.10828955e-01,
        3.14759949e-01]),
 np.array([2.09274789e-04, 5.70991281e-05, 9.47387269e-01, 2.11830441e-01,
        2.58944415e-01]),
 np.array([2.06435007e-04, 5.10299405e-05, 9.49167588e-01, 2.12162859e-01,
        3.31884807e-01]),
 np.array([1.99986280e-04, 5.14765686e-05, 9.49834695e-01, 2.12835590e-01,
        2.71392958e-01]),
 np.array([2.13263178e-04, 5.15006629e-05, 9.47973599e-01, 2.10340277e-01,
        2.78024268e-01]),
 np.array([2.13263178e-04, 5.15006629e-05, 9.47973599e-01, 2.10340277e-01,
        2.78024268e-01])]

In [ ]:
result

In [ ]:
df_res_minima = []
mae_minima = []
rmse_minima = []

for params_minimum in intermediate_solutions:
    mae_minima.append([])
    rmse_minima.append([])

    df_concat = pd.DataFrame()
    for triplet in triplets:
        df_res = simulation_function(params_minimum, triplet)

        triplet['df_res'] = df_res

        df_res['Qdot_experimental'] = triplet['steadyStateConditions_df']['Qdot']
        df_concat = pd.concat([df_concat, df_res], axis=0)
        mae_minima[-1].append(sklearn.metrics.mean_absolute_error(df_res['Qdot_tube_fluid'], df_res['Qdot_experimental']))
        rmse_minima[-1].append(np.sqrt(((df_res['Qdot_tube_fluid'] - triplet['steadyStateConditions_df']['Qdot']) ** 2).mean()))
    df_res_minima.append(df_concat)

In [ ]:
mae_minima

In [ ]:
triplets[0]['df_res']

In [ ]:
fig, ax = plt.subplots(figsize=(rconfig.wcol_in/2, rconfig.hfig(0.2, unit="in")))

u_range = [(0,1), (1,2), (2,4)]
k = 0

colors = ['olive', 'blue', 'orange']
colors_lighter = [rconfig.get_hex_code(color, 70) for color in colors]
colors = [rconfig.get_hex_code(color, 100) for color in colors]
linestyles = ['-', '--',  (5, (10, 3)), '-.', (0, (1, 1)), (0, (3, 5, 1, 5))]
markers = ['o', 's', 'd']

u_list = list(triplets[0]['steadyStateConditions_df']['u'].unique())

for i, (u_min, u_max) in enumerate(u_range):
    df_f = triplets[k]['df_res'].loc[(u_min <= triplets[k]['df_res']['u']) & (triplets[k]['df_res']['u'] < u_max)].copy()
    df_f.sort_values('T_m - T_amb', inplace=True)
    
    ax.plot(df_f['T_m - T_amb'], df_f['Qdot_tube_fluid'], label='Numerical model', color = colors[i], linestyle=linestyles[i])
    ax.scatter(df_f['T_m - T_amb'], df_f['Qdot_experimental'], label='Experimental data', color=colors[i], marker = markers[i])

# Legend
# for i, (u_min, u_max) in enumerate(u_range):
#     df_f = triplets[k]['df_res'].loc[(u_min <= triplets[k]['df_res']['u']) & (triplets[k]['df_res']['u'] < u_max)].copy()
#     df_f.sort_values('T_m - T_amb', inplace=True)
    
#     ax.plot([], [], label=f'Model - u = {str(u_list[i])} m/s', color = colors[i], linestyle=linestyles[i])
#     ax.scatter([], [], label=f'Experiment - u = {str(u_list[i])} m/s', color=colors[i], marker = markers[i])

# ax.set_xticks(np.arange(0, max(df_f['T_m - T_amb']) + 10, 5))
# ax.set_yticks(np.arange(0,800+10, 200))

# Move the x-axis and y-axis to the center
ax.spines['left'].set_position('zero')
ax.spines['bottom'].set_position('zero')

# Remove top and right spines (the frame)
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)

# Keep ticks at the original margin positions
ax.xaxis.set_ticks_position('bottom')  # Ticks stay at the bottom
ax.yaxis.set_ticks_position('left')    # Ticks stay at the left

ax.tick_params(axis='x', which='major', pad=35)
# ax.tick_params(axis='x', which='major', pad=120) # for k=1 or 3
ax.tick_params(axis='y', which='major', pad=23)

plt.grid(linewidth=0.5, color=rconfig.get_hex_code('grey', 70))

ax.set_xlabel(r'$T_m - T_{amb}$ [K]')
ax.set_ylabel(r'$\dot{Q}$ [W]',
              labelpad = 2 
              )

# ax.legend(loc='lower center', ncol=3, frameon=False)

# Legend
# plt.axis('off')  # Turns off the axes
# ax.legend(loc='center', ncol=3, frameon=False)  # Adjust the legend location and style

ax.set_title('S-S\&T-NI - SK',loc = 'center', pad = 10, fontsize = 14)

plt.show()

In [ ]:
triplets[0]['df_res']['relative error'] = (triplets[0]['df_res']['Qdot_tube_fluid'] - triplets[0]['df_res']['Qdot_experimental']) / triplets[0]['df_res']['Qdot_experimental']
sns.histplot(triplets[0]['df_res']['relative error'], bins=10)

In [ ]:
triplets[k]['df_res'][['T_m - T_amb','u']]

In [ ]:
fig, ax = plt.subplots(figsize=(rconfig.wcol_in/2, rconfig.hfig(0.2, unit="in")))

u_range = [(0,1), (1,2), (2,4)]
k = 1

colors = ['olive', 'blue', 'orange']
colors_lighter = [rconfig.get_hex_code(color, 70) for color in colors]
colors = [rconfig.get_hex_code(color, 100) for color in colors]
linestyles = ['-', '--',  (5, (10, 3)), '-.', (0, (1, 1)), (0, (3, 5, 1, 5))]
markers = ['o', 's', 'd', 'X', 'v', '^']

u_list = list(triplets[0]['steadyStateConditions_df']['u'].unique())

for i, (u_min, u_max) in enumerate(u_range):
    df_f = triplets[k]['df_res'].loc[(u_min <= triplets[k]['df_res']['u']) & (triplets[k]['df_res']['u'] < u_max)].copy()
    df_f.sort_values('T_m - T_amb', inplace=True)
    
    ax.plot(df_f['T_m - T_amb'], df_f['Qdot_tube_fluid'], label='Numerical model', color = colors[i], linestyle=linestyles[i])
    ax.scatter(df_f['T_m - T_amb'], df_f['Qdot_experimental'], label='Experimental data', color=colors[i], marker = markers[i])

# Legend
# for i, (u_min, u_max) in enumerate(u_range):
#     df_f = triplets[k]['df_res'].loc[(u_min <= triplets[k]['df_res']['u']) & (triplets[k]['df_res']['u'] < u_max)].copy()
#     df_f.sort_values('T_m - T_amb', inplace=True)
    
#     ax.plot([], [], label=f'Model - u = {str(u_list[i])} m/s', color = colors[i], linestyle=linestyles[i])
#     ax.scatter([], [], label=f'Experiment - u = {str(u_list[i])} m/s', color=colors[i], marker = markers[i])

# ax.set_xticks(np.arange(0, max(df_f['T_m - T_amb']) + 10, 5))
# ax.set_yticks(np.arange(0,800+10, 200))

# Move the x-axis and y-axis to the center
# ax.spines['left'].set_position('zero')
ax.spines['bottom'].set_position('zero')

# Remove top and right spines (the frame)
ax.spines['top'].set_visible(False)
ax.spines['left'].set_visible(False)

# Keep ticks at the original margin positions
ax.xaxis.set_ticks_position('bottom')  # Ticks stay at the bottom
ax.yaxis.set_ticks_position('right')    # Ticks stay at the left

ax.yaxis.set_label_position("right")

# ax.tick_params(axis='x', which='major', pad=35)
# ax.tick_params(axis='x', which='major', pad=120) # for k=1 or 3
# ax.tick_params(axis='y', which='major', pad=23)

plt.grid(linewidth=0.5, color=rconfig.get_hex_code('grey', 70))

ax.set_xlabel(r'$T_m - T_{amb}$ [K]')
ax.set_ylabel(r'$\dot{Q}$ [W]',
              labelpad = 2,
              loc='center'
              )

ax.set_ylim(0,325)

# ax.legend(loc='lower center', ncol=3, frameon=False)

# Legend
# plt.axis('off')  # Turns off the axes
# ax.legend(loc='center', ncol=3, frameon=False)  # Adjust the legend location and style

ax.set_title('S-S\&T-NI - dark',loc = 'center', pad = 10, fontsize = 14)

plt.show()

In [ ]:
triplets[1]['df_res']['relative error'] = (triplets[0]['df_res']['Qdot_tube_fluid'] - triplets[0]['df_res']['Qdot_experimental']) / triplets[0]['df_res']['Qdot_experimental']
sns.histplot(triplets[1]['df_res']['relative error'], bins=10)

In [ ]:
fig, ax = plt.subplots(figsize=(rconfig.wcol_in/2, rconfig.hfig(0.2, unit="in")))

u_range = [(0,1), (1,2), (2,3)]
k = 2

colors = ['olive', 'blue', 'orange']
colors_lighter = [rconfig.get_hex_code(color, 70) for color in colors]
colors = [rconfig.get_hex_code(color, 100) for color in colors]
linestyles = ['-', '--',  (5, (10, 3)), '-.', (0, (1, 1)), (0, (3, 5, 1, 5))]
markers = ['o', 's', 'd']

u_list = list(triplets[0]['steadyStateConditions_df']['u'].unique())

for i, (u_min, u_max) in enumerate(u_range):
    df_f = triplets[k]['df_res'].loc[(u_min <= triplets[k]['df_res']['u']) & (triplets[k]['df_res']['u'] < u_max)].copy()
    df_f.sort_values('T_m - T_amb', inplace=True)
    
    ax.plot(df_f['T_m - T_amb'], df_f['Qdot_tube_fluid'], label='Numerical model', color = colors[i], linestyle=linestyles[i])
    ax.scatter(df_f['T_m - T_amb'], df_f['Qdot_experimental'], label='Experimental data', color=colors[i], marker = markers[i])

# Legend
# for i, (u_min, u_max) in enumerate(u_range):
#     df_f = triplets[k]['df_res'].loc[(u_min <= triplets[k]['df_res']['u']) & (triplets[k]['df_res']['u'] < u_max)].copy()
#     df_f.sort_values('T_m - T_amb', inplace=True)
    
#     ax.plot([], [], label=f'Model - u = {str(u_list[i])} m/s', color = colors[i], linestyle=linestyles[i])
#     ax.scatter([], [], label=f'Experiment - u = {str(u_list[i])} m/s', color=colors[i], marker = markers[i])

# ax.set_xticks(np.arange(0, max(df_f['T_m - T_amb']) + 10, 5))
ax.set_yticks(np.arange(0,800+10, 200))

# Move the x-axis and y-axis to the center
ax.spines['left'].set_position('zero')
ax.spines['bottom'].set_position('zero')

# Remove top and right spines (the frame)
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)

# Keep ticks at the original margin positions
ax.xaxis.set_ticks_position('bottom')  # Ticks stay at the bottom
ax.yaxis.set_ticks_position('left')    # Ticks stay at the left

# ax.tick_params(axis='x', which='major', pad=35)
# ax.tick_params(axis='x', which='major', pad=120) # for k=1 or 3
ax.tick_params(axis='y', which='major', pad=23)

plt.grid(linewidth=0.5, color=rconfig.get_hex_code('grey', 70))

ax.set_xlabel(r'$T_m - T_{amb}$ [K]')
ax.set_ylabel(r'$\dot{Q}$ [W]',
            #   labelpad = 2 
              )

# ax.legend(loc='lower center', ncol=3, frameon=False)

# Legend
# plt.axis('off')  # Turns off the axes
# ax.legend(loc='center', ncol=3, frameon=False)  # Adjust the legend location and style

ax.set_title('S-S\&T-I - SK',loc = 'center', pad = 10, fontsize = 14)

plt.show()

In [ ]:
triplets[2]['df_res'][['u','Qdot_tube_fluid', 'Qdot_experimental']]

In [ ]:
fig, ax = plt.subplots(figsize=(rconfig.wcol_in/2, rconfig.hfig(0.2, unit="in")))

u_range = [(0,1), (1,2), (2,4)]
k = 3

colors = ['olive', 'blue', 'orange']
colors_lighter = [rconfig.get_hex_code(color, 70) for color in colors]
colors = [rconfig.get_hex_code(color, 100) for color in colors]
linestyles = ['-', '--',  (5, (10, 3)), '-.', (0, (1, 1)), (0, (3, 5, 1, 5))]
markers = ['o', 's', 'd']

u_list = list(triplets[0]['steadyStateConditions_df']['u'].unique())

for i, (u_min, u_max) in enumerate(u_range):
    df_f = triplets[k]['df_res'].loc[(u_min <= triplets[k]['df_res']['u']) & (triplets[k]['df_res']['u'] < u_max)].copy()
    df_f.sort_values('T_m - T_amb', inplace=True)
    
    ax.plot(df_f['T_m - T_amb'], df_f['Qdot_tube_fluid'], label='Numerical model', color = colors[i], linestyle=linestyles[i])
    ax.scatter(df_f['T_m - T_amb'], df_f['Qdot_experimental'], label='Experimental data', color=colors[i], marker = markers[i])

# Legend
# for i, (u_min, u_max) in enumerate(u_range):
#     df_f = triplets[k]['df_res'].loc[(u_min <= triplets[k]['df_res']['u']) & (triplets[k]['df_res']['u'] < u_max)].copy()
#     df_f.sort_values('T_m - T_amb', inplace=True)
    
#     ax.plot([], [], label=f'Model - u = {str(u_list[i])} m/s', color = colors[i], linestyle=linestyles[i])
#     ax.scatter([], [], label=f'Experiment - u = {str(u_list[i])} m/s', color=colors[i], marker = markers[i])

# ax.set_xticks(np.arange(0, max(df_f['T_m - T_amb']) + 10, 5))
# ax.set_yticks(np.arange(0,800+10, 200))

# Move the x-axis and y-axis to the center
ax.spines['left'].set_position('zero')
ax.spines['bottom'].set_position('zero')

# Remove top and right spines (the frame)
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)

# Keep ticks at the original margin positions
ax.xaxis.set_ticks_position('bottom')  # Ticks stay at the bottom
ax.yaxis.set_ticks_position('left')    # Ticks stay at the left

# ax.tick_params(axis='x', which='major', pad=35)
ax.tick_params(axis='x', which='major', pad=120) # for k=1 or 3
ax.tick_params(axis='y', which='major', pad=23)

plt.grid(linewidth=0.5, color=rconfig.get_hex_code('grey', 70))

ax.set_xlabel(r'$T_m - T_{amb}$ [K]')
ax.set_ylabel(r'$\dot{Q}$ [W]',
              labelpad = 2 
              )

# ax.legend(loc='lower center', ncol=3, frameon=False)

# Legend
# plt.axis('off')  # Turns off the axes
# ax.legend(loc='center', ncol=3, frameon=False)  # Adjust the legend location and style

ax.set_title('S-S\&T-I - dark',loc = 'center', pad = 10, fontsize = 14)

plt.show()

In [ ]:
triplets[k]['df_res'][['Qdot_tube_fluid', 'Qdot_experimental']]

In [ ]:
k=3

triplets[k]['df_res']['relative error'] = (triplets[k]['df_res']['Qdot_tube_fluid'] - triplets[k]['df_res']['Qdot_experimental']) / triplets[k]['df_res']['Qdot_experimental']
sns.histplot(triplets[k]['df_res']['relative error'], bins=10)